https://stockant.tistory.com/category/%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%98%EB%B0%8D/%EC%98%81%ED%99%94%20TMDB%20API

In [5]:
import requests

In [27]:
# TMDB API 키
TMDB_API_KEY = "df9a0caaf2a07ee6babd7024a6accaf8"

In [26]:
url = f"https://api.themoviedb.org/3/genre/movie/list?api_key={TMDB_API_KEY}"

response = requests.get(url)
data = response.json()
genres = data.get("genres", [])

GENRES = {}
for genre in genres:
    GENRES[genre['name']] = genre['id']
    
    print(genre['name'], GENRES[genre['name']])

Action 28
Adventure 12
Animation 16
Comedy 35
Crime 80
Documentary 99
Drama 18
Family 10751
Fantasy 14
History 36
Horror 27
Music 10402
Mystery 9648
Romance 10749
Science Fiction 878
TV Movie 10770
Thriller 53
War 10752
Western 37


In [28]:
EMOTION_TO_GENRE = {
    '기쁨': 35,  # Comedy
    '슬픔': 18,  # Drama
    '분노': 53,  # Thriller
    '불안': 27,  # Horror
    '상처': 80,  # Crime
    '당황': 28,  # Action
    '중립': 10751,  # Family
}

In [19]:
def get_recommendations(emotion, result_num=10, api_key=TMDB_API_KEY):
    # 감정 매핑 확인
    genre_id = EMOTION_TO_GENRE.get(emotion)
    if not genre_id:
        return f"'{emotion}'에 해당하는 추천 장르가 없습니다. 감정을 다시 입력해주세요."

    # TMDB Discover API 호출
    url = f"https://api.themoviedb.org/3/discover/movie"
    params = {
        "api_key": api_key,
        "with_genres": genre_id,
        "sort_by": "popularity.desc",  # 인기 순으로 정렬
        "language": "ko-KR",          # 한국어 결과
        "vote_average.gte": 7.0,      # 평점 7 이상
    }

    response = requests.get(url, params=params)
    if response.status_code != 200:
        return f"TMDB API 호출 실패: {response.status_code}"

    data = response.json()
    results = data.get("results", [])

    if not results:
        return f"'{emotion}'에 맞는 추천 콘텐츠를 찾을 수 없습니다."

    # 추천 콘텐츠 추출
    recommendations = []
    for movie in results[:result_num]:  # 상위 10개만 추출
        recommendations.append({
            "title": movie.get("title"),
            "overview": movie.get("overview"),
            "vote_average": movie.get("vote_average"),
            "release_date": movie.get("release_date"),
        })

    return recommendations

In [30]:
user_emotion = input("현재 감정을 입력하세요 (기쁨, 슬픔, 분노, 불안, 상처, 당황, 중립): ")
recommendations = get_recommendations(user_emotion, 3)

if isinstance(recommendations, str):
    print(recommendations)  # 에러 메시지 출력
else:
    print(f"'{user_emotion}'에 맞는 추천 콘텐츠:")
    for idx, movie in enumerate(recommendations, 1):
        print(f"\n{idx}. 제목: {movie['title']}")
        print(f"   개봉일: {movie['release_date']}")
        print(f"   평점: {movie['vote_average']}")
        print(f"   줄거리: {movie['overview']}")

'당황'에 맞는 추천 콘텐츠:

1. 제목: 수퍼 소닉 3
   개봉일: 2024-12-19
   평점: 7.7
   줄거리: 너클즈, 테일즈와 함께 평화로운 일상을 보내던 초특급 히어로 소닉. 연구 시설에 50년간 잠들어 있던 사상 최강의 비밀 병기 "섀도우"가 탈주하자, 세계 수호 통합 부대(약칭 세.수.통)에 의해 극비 소집된다. 소중한 것을 잃은 분노와 복수심에 불타는 섀도우는 소닉의 초고속 스피드와 너클즈의 최강 펀치를 단 단숨에 제압해버린다. 세상을 지배하려는 닥터 로보트닉과 그의 할아버지 제럴드 박사는 섀도우의 엄청난 힘 카오스 에너지를 이용해 인류를 정복하려고 하는데…

2. 제목: 레드 원
   개봉일: 2024-10-31
   평점: 7.041
   줄거리: 크리스마스 D-1, 철통같은 보안을 뚫고 코드명 '레드 원' 산타클로스가 납치되고 크리스마스가 사라질 위기에 처했다! '레드 원'을 찾기 위해 사령관 '칼럼 드리프트'는 산타클로스 따위는 없다고 믿는 현상금 사냥꾼 '잭 오말리'와 협력하기로 한다. 시작부터 삐그덕 거리는 이들 앞에 크리스마스의 존재를 위협하는 위험천만한 적들이 나타나는데…

3. 제목: 서브스턴스
   개봉일: 2024-09-07
   평점: 7.2
   줄거리: 더 나은 버전의 당신을 꿈꿔본 적 있나요? 당신의 인생을 바꿔줄 신제품 ‘서브스턴스’. ‘서브스턴스’는 또 다른 당신을 만들어냅니다. 새롭고, 젊고, 더 아름답고, 더 완벽한 당신을. 단 한가지 규칙, 당신의 시간을 공유하면 됩니다. 당신을 위한 일주일, 새로운 당신을 위한 일주일, 각각 7일간의 완벽한 밸런스. 쉽죠? 균형을 존중한다면… 무엇이 잘못될 수 있을까요?
